In [ ]:
from selenium import webdriver
import json
import time


# 设置保存为pdf的参数
appState = """{
    "recentDestinations": [
        {
            "id": "Save as PDF",
            "origin": "local"
        }
    ],
    "mediaSize": {
        "height_microns": 279400,
        "name": "NA_LETTER",
        "width_microns": 215900,
        "custom_display_name": "Letter"
    },
    "selectedDestinationId": "Save as PDF",
    "version": 2,
    "isHeaderFooterEnabled": false
}"""

# 设置pdf的保存路径
pdf_save_path = "D:\\Download\\pdf\\"

appState = json.loads(appState)
profile = {'printing.print_preview_sticky_settings.appState': json.dumps(appState),'savefile.default_directory': pdf_save_path,'download.default_directory': pdf_save_path}
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('prefs', profile)

# 该参数必须存在，不设置会无法使用打印，保存全屏图片等功能
chrome_options.add_argument('--kiosk-printing')

# 用指定路径的ChromeDriver驱动去打开一个浏览器窗口
driver = webdriver.Chrome(executable_path="C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe",options=chrome_options)

# 登录搜狗微信
driver.get("https://weixin.sogou.com/")
time.sleep(3)
driver.find_elements_by_xpath('//a[@id="loginBtn"]')[0].click()

In [ ]:
def scroll_to_bottom(driver):
    js = "return action=document.body.scrollHeight"
    # 初始化现在滚动条所在高度为0
    height = 0
    # 当前窗口总高度
    new_height = driver.execute_script(js)

    while height < new_height:
        # 将滚动条调整至页面底部
        for i in range(height, new_height, 100):
            driver.execute_script('window.scrollTo(0, {})'.format(i))
            time.sleep(0.5)
        height = new_height
        time.sleep(2)
        new_height = driver.execute_script(js)
        
def get_pdf(n):
    for i in range(n):
        for j in range(10):
            title = driver.find_elements_by_xpath('//a[@uigs="article_title_{}"]'.format(j))[0].text.replace(" ","-") 
            driver.find_elements_by_xpath('//a[@uigs="article_title_{}"]'.format(j))[0].click()
            driver.switch_to.window(driver.window_handles[1])
            time.sleep(2)
            scroll_to_bottom(driver)
            driver.execute_script('document.title="{}";window.print();'.format(title))
            time.sleep(2)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(2)

def pdf_to_pages_photo(pdfPath, imagePath, split_name):
    
    pdf_context = fitz.open(pdfPath)
    for pg in range(pdf_context.pageCount):
        page = pdf_context[pg]
        rotate = int(0)
        # 每个尺寸的缩放系数为1.3，这将为我们生成分辨率提高2.6的图像。
        # 此处若是不做设置，默认图片大小为：792X612, dpi=96
        zoom_x = 1.33333333  # (1.33333333-->1056x816)   (2-->1584x1224)
        zoom_y = 1.33333333
        mat = fitz.Matrix(zoom_x, zoom_y).preRotate(rotate)
        pix = page.getPixmap(matrix=mat, alpha=False)
        if not os.path.exists(imagePath):  # 判断存放图片的文件夹是否存在
            os.makedirs(imagePath)  # 若图片文件夹不存在就创建
        pix.writePNG(imagePath + '{}{}.png'.format(split_name,str(pg+1)))  # 将图片写入指定的文件夹内

def pages_photo_APhoto(path,split_photo_name,title):
    page_photo_list = []
    image_list = []
    
    page_photo_list = glob.glob('{}{}*.png'.format(path,split_photo_name))
    page_photo_list = [fn.replace(path,'') for fn in page_photo_list]
    # 文件夹名 按数字排序
    page_photo_list.sort(key=lambda x: int(x[len(split_photo_name):-4]))
    
    for pphoto in page_photo_list:
        image_list.append(Image.open(path + os.sep + pphoto))
    
    width = 0
    height = 0
    for img in image_list:
        # 单幅图像尺寸
        w, h = img.size
        height += h
        # 取最大的宽度作为拼接图的宽度
        width = max(width, w)

    # 创建空白长图
    result = Image.new(image_list[0].mode, (width, height), 0xffffff)
    # 拼接图片
    height = 0
    for img in image_list:
        w, h = img.size
        # 图片水平居中
        result.paste(img, box=(round(width / 2 - w / 2), height))
        height += h
    # 保存图片
    result.save('{}{}.png'.format(path,title),quality=95)
    
    # 删除那些片段化的图片，因为已经合成了长图片了
    for pphoto in page_photo_list:
        os.remove(path+pphoto)
    
    time.sleep(1)
        
        
if __name__ == "__main__":
    
    # 1、文本框中输入你要收集的学习文章的搜索关键字“XXX”,本次案例设置搜索词为selenium
    query = driver.find_element_by_name("query")
    query.send_keys("selenium")

    # 2、点击搜索按钮
    time.sleep(1)
    driver.find_elements_by_xpath('//input[@uigs="search_article"]')[0].click()
    
    # 3、打印了所有的目标pdf,最多100页，如果不先扫码登录，最多就只能爬取10页
    get_pdf(100)
    
    # 4、获取刚刚爬取的所有pdf的存放路径
    list_pdf = glob.glob(pdf_save_path+'*.pdf')
    
    # 5、定义储存图片的目录
    imagePath = 'D:\\Download\\photo\\'
    
    for pdf in list_pdf:
        # 6、循环得到的每一个PDF地址
        pdfPath = pdf
        
        # 7、将pdf转化为图片，是按页转换
        split_photo_name = 'images_'
        pdf_to_pages_photo(pdfPath, imagePath, split_photo_name)

        # 8、将按页转换的图片转化为一张长图片，并删除多余的图片
        title = os.path.split(pdf)[1].replace('.pdf','')
        pages_photo_APhoto(imagePath,split_photo_name,title)